In [1]:
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import numpy as np
import pandas as pd
import dask.array as da
#from tensorflow.python.client import timeline
import dask.dataframe as dd
import scipy.stats as st
import mirsig.util.utility as ut
import mirsig
import re

from sklearn.metrics import precision_recall_curve, precision_recall_fscore_support, f1_score
%matplotlib inline

In [2]:
# row major
# n = numvars
# nv = number of voters, algorithms
# mats = list of all matricies
def borda (mats, n, nv):
    
    for v in range (nv):
        mats[v] = mats[v].argsort(axis = None)
        
    borda_points = np.zeros((n*n))
    for i in range(n*n):
        # i is the current rank star between 0-15
        # rank pint is i/highest rank possible (n*n-1)
        
        for v in range (nv):
            
            borda_points[(mats[v])[i]] = borda_points[(mats[v])[i]] + (i/(n*n-1))
        
         
    return ((borda_points/nv).reshape(n,n))

# Test

In [3]:
# dream_df, column_names = ut.get_data("data/dream4/insilico_size100_2_multifactorial.tsv", sep ='\t')[0:2]
# dream_df, column_names = ut.get_data("PD_data_GEO.csv")[0:2]

In [4]:
dream_df = pd.read_csv("ADgeoSelectedGene.csv", index_col=0)

In [5]:
dream_df.head()

,GSM2233534,GSM2233535,GSM2233537,GSM2233538,GSM2233539,GSM2233540,GSM2233541,GSM2233542,GSM2233544,GSM2233547,...,GSM2235488,GSM2235490,GSM2235495,GSM2235501,GSM2235503,GSM2235504,GSM2235512,GSM2235519,GSM2235521,GSM2235522
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
79873,2.578980,2.846728,2.810996,3.028982,2.952253,2.610754,2.453150,2.946256,3.001733,2.318878,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
152579,4.791409,4.421290,5.032810,5.456838,4.824444,4.005867,4.909454,5.389630,4.921789,3.837278,...,3.323182,3.677287,3.85236,3.784356,3.740637,3.662195,3.730365,3.438008,3.530107,3.452541
79890,3.046621,2.899805,3.439115,2.650094,3.100683,3.015600,2.929233,2.493586,2.947012,3.337367,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25,6.830657,6.409269,6.217309,6.050916,6.406295,6.679915,6.938314,5.905703,5.855430,6.455337,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1051,8.739209,8.808062,10.541026,9.204943,9.228715,9.158074,9.157727,8.532056,9.248596,10.393923,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
column_names = list(dream_df.index)
dream_df = dream_df.T

In [7]:
column_names

[79873,
 152579,
 79890,
 25,
 1051,
 28,
 23588,
 23595,
 10284,
 124976,
 51248,
 3123,
 5173,
 23607,
 135228,
 5199,
 2128,
 5201,
 254042,
 30815,
 102,
 4199,
 10347,
 114796,
 4204,
 114798,
 114799,
 8301,
 114801,
 30834,
 4208,
 5245,
 10369,
 58503,
 374920,
 2185,
 1163,
 64651,
 3213,
 100289678,
 402573,
 10384,
 1173,
 51350,
 259230,
 100128927,
 1184,
 2206,
 57506,
 123041,
 54436,
 3237,
 1191,
 100129961,
 25769,
 25771,
 2220,
 7337,
 177,
 84148,
 10425,
 25787,
 144577,
 25796,
 51398,
 6345,
 54474,
 3276,
 9420,
 51412,
 10452,
 84186,
 5348,
 474344,
 25833,
 10476,
 10477,
 240,
 100505841,
 5366,
 5376,
 29958,
 3339,
 1293,
 101928207,
 149775,
 274,
 275,
 51477,
 102723864,
 57633,
 22820,
 80167,
 2348,
 25906,
 55607,
 146754,
 323,
 6474,
 56652,
 25937,
 56660,
 341,
 254295,
 8537,
 128346,
 3418,
 54620,
 56667,
 348,
 5471,
 147808,
 1378,
 51555,
 116068,
 356,
 26986,
 100130155,
 84331,
 134510,
 51567,
 25976,
 114041,
 1404,
 85377,
 140679,
 

In [8]:
# run mrnetb befor clr!!
mrnetb_res= mirsig.mrnetb_cuda(dream_df)

Mrnetb for : 307
21.040469s sec


In [9]:
mrnetb_res

,0,1,2,3,4,5,6,7,8,9,...,297,298,299,300,301,302,303,304,305,306
0,1.048264e-04,1.504021e-02,1.801122e-03,1.344495e-02,9.581354e-04,1.462232e-03,1.050121e-03,6.013495e-03,4.971277e-03,7.797020e-03,...,1.623134e-04,2.399186e-03,4.734876e-04,2.417494e-02,3.061222e-03,3.279940e-03,6.507642e-04,8.094959e-07,5.599645e-02,7.740056e-03
1,1.504021e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.183067e-06,...,0.000000e+00,7.137425e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.252185e-08,9.047279e-08,8.547150e-07,1.985414e-06
2,1.801122e-03,0.000000e+00,0.000000e+00,1.307078e-06,2.925924e-06,1.366741e-06,1.748214e-06,1.069075e-06,1.721889e-06,0.000000e+00,...,1.754133e-06,3.129148e-07,1.439186e-06,8.620725e-07,1.913570e-06,1.284973e-06,3.130360e-07,1.119222e-06,0.000000e+00,0.000000e+00
3,1.344495e-02,0.000000e+00,1.307078e-06,0.000000e+00,1.190334e-06,3.242693e-07,8.738047e-07,3.787186e-07,1.993896e-06,0.000000e+00,...,4.411071e-07,0.000000e+00,4.184146e-07,1.055180e-06,1.796787e-06,1.267799e-06,8.746910e-08,1.435695e-07,0.000000e+00,0.000000e+00
4,9.581354e-04,0.000000e+00,2.925924e-06,1.190334e-06,0.000000e+00,1.579618e-06,2.340799e-06,1.448978e-06,1.679146e-06,0.000000e+00,...,1.979129e-06,1.197511e-07,1.394449e-06,8.030407e-07,2.200695e-06,1.603407e-06,5.059911e-07,8.757861e-07,0.000000e+00,0.000000e+00
5,1.462232e-03,0.000000e+00,1.366741e-06,3.242693e-07,1.579618e-06,0.000000e+00,1.314067e-06,2.790039e-07,5.955901e-07,0.000000e+00,...,2.439485e-06,3.326706e-07,2.192849e-06,4.472558e-07,1.028685e-06,8.047988e-07,8.998942e-07,1.506215e-06,0.000000e+00,0.000000e+00
6,1.050121e-03,0.000000e+00,1.748214e-06,8.738047e-07,2.340799e-06,1.314067e-06,0.000000e+00,2.293447e-06,1.795904e-06,0.000000e+00,...,2.266910e-06,3.968702e-07,1.196687e-06,7.589371e-07,2.271640e-06,2.596851e-06,1.001623e-06,1.069673e-06,0.000000e+00,0.000000e+00
7,6.013495e-03,0.000000e+00,1.069075e-06,3.787186e-07,1.448978e-06,2.790039e-07,2.293447e-06,0.000000e+00,1.579091e-06,0.000000e+00,...,1.033621e-06,0.000000e+00,2.453577e-07,2.355511e-07,1.784335e-06,1.636709e-06,1.282902e-07,1.933876e-07,0.000000e+00,0.000000e+00
8,4.971277e-03,0.000000e+00,1.721889e-06,1.993896e-06,1.679146e-06,5.955901e-07,1.795904e-06,1.579091e-06,0.000000e+00,0.000000e+00,...,1.020021e-06,0.000000e+00,6.767087e-07,9.429727e-07,2.603018e-06,1.390550e-06,1.600114e-07,2.959868e-07,0.000000e+00,0.000000e+00
9,7.797020e-03,2.183067e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,9.399656e-08,5.749740e-07,2.896260e-07,0.000000e+00,0.000000e+00,0.000000e+00,6.088868e-07,5.914964e-07,1.154859e-06,3.018829e-06


In [10]:
clr_res = mirsig.clr(dream_df.values,128,16)

In [11]:
genie_res = mirsig.parallel_genie(dream_df)

100%

In [12]:
genie_no_dir = pd.DataFrame(np.maximum(genie_res.values, genie_res.values.T)) 

In [13]:
spearman_res = mirsig.spearman_df(dream_df)

In [14]:
pearson_res = mirsig.pearson_df(dream_df)

In [15]:
#mrnetb_res= mirsig.mrnetb_cuda()
pearson_res = pearson_res.abs()

In [16]:
spearman_res = spearman_res.abs()

In [17]:
kendall_res = mirsig.kendal_taub_cuda(dream_df)

Kendall for : 307
0.091925s sec


In [18]:
borda_res = borda([mrnetb_res.values,
                   clr_res,
                   genie_res.values,
                   kendall_res.values,
                   spearman_res.values,
                   pearson_res.values],
                   pearson_res.shape[0], 6)
np.fill_diagonal(borda_res, 0)

In [19]:
np.save('data', borda_res)

In [20]:
# borda_res = np.load('intermidiate/data.npy')

In [21]:
borda_res.shape

(307, 307)

In [22]:
for i in([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]):
    print(np.sum((borda_res > i) & (borda_res<i+0.1)))
print(np.sum((borda_res > 0.95) & (borda_res<1)))
print(np.sum((borda_res > 0.999) & (borda_res<1)))

8513
13270
14956
10253
9726
10668
10855
10705
3183
434
0


In [23]:
dS = pd.DataFrame (borda_res, columns= column_names, index= column_names )
edgeList = dS.stack().reset_index()
edgeList=edgeList[edgeList[0]>0.9]
edgeList[['level_0','level_1'] ].to_csv('AD_Selected_network_edgelist_90.tsv', sep='\t', header=False, index=False)

In [24]:
dS = pd.DataFrame (borda_res, columns= column_names, index= column_names )
edgeList = dS.stack().reset_index()
edgeList=edgeList[edgeList[0]>0.95]
edgeList[['level_0','level_1'] ].to_csv('AD_Selected_network_edgelist_95.tsv', sep='\t', header=False, index=False)

In [25]:
dS = pd.DataFrame (borda_res, columns= column_names, index= column_names )
edgeList = dS.stack().reset_index()
edgeList=edgeList[edgeList[0]>0.975]
edgeList[['level_0','level_1'] ].to_csv('AD_Selected_network_edgelist_975.tsv', sep='\t', header=False, index=False)

In [26]:
dS = pd.DataFrame (borda_res, columns= column_names, index= column_names )
edgeList = dS.stack().reset_index()
edgeList=edgeList[edgeList[0]>0.99]
edgeList[['level_0','level_1'] ].to_csv('AD_Selected_network_edgelist_99.tsv', sep='\t', header=False, index=False)